# BASELINE 1

Pedir para um LLM gerar perguntas e respostas fornecendo somente um conjunto aleatório de artigos.

- Para facilitar vamos lidar inicialmente com o Direito da Família (Parte Especial Livro IV) e Direito de Sucessões (Parte Especial Livro V)

Obs: Como não temos a métrica de avaliação implementada vamos gerar 25 perguntas
para avaliação qualitativa

# Imports e Setup de Ambiente

In [1]:
!pip install langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 8.8 MB/s eta 0:00:00


In [2]:
import json
import random
import numpy as np
import pandas as pd
from getpass import getpass
from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Set the random seed
random.seed(42)

## Load Código Civil

In [3]:
with open("codigo_civil.json", "r") as f:
    data = json.load(f)

In [4]:
def collect_artigo_and_header(data):
    result = {'articles': [], 'contexts': []}

    def traverse(data):
        if isinstance(data, dict):
            for key, value in data.items():
                if key == 'artigos' and value:
                    result['articles'].append(value)
                if key == 'header' and value:
                    result['contexts'].append(value)
                # Recursive call to handle nested dictionaries
                if isinstance(value, dict):
                    traverse(value)
                elif isinstance(value, list):
                    traverse(value)
        elif isinstance(data, list):
            for item in data:
                # Recursive call to handle lists of dictionaries
                traverse(item)

    traverse(data)
    return result

result = collect_artigo_and_header(data['parte especial']['LIVRO IV'])
iv_context = ''.join(result['contexts'])
iv_articles = result['articles']

result = collect_artigo_and_header(data['parte especial']['LIVRO V'])
v_context = ''.join(result['contexts'])
v_articles = result['articles']

In [5]:
articles = [article for articles in iv_articles+v_articles for article in articles if article]
contexts = '\n\n\n'.join([iv_context, v_context])

In [6]:
len(articles)

518

In [7]:
ids = [int(article.split('. ')[1].replace('.', '').split('-')[0]) for article in articles]
check_ids = np.arange(1511, 2028)

list(set(check_ids) - set(ids))

[]

In [8]:
[art for art in articles if len(art.split('. ')[1].split('-')) > 1] # Lei que tem a mais pois se encontra em outro capitulo

['Art. 1.783-A.  A tomada de decisão apoiada é o processo pelo qual a pessoa com deficiência elege pelo menos 2 (duas) pessoas idôneas, com as quais mantenha vínculos e que gozem de sua confiança, para prestar-lhe apoio na tomada de decisão sobre atos da vida civil, fornecendo-lhes os elementos e informações necessários para que possa exercer sua capacidade.      (Incluído pela Lei nº 13.146, de 2015)   (Vigência)\n\n§ 1 o Para formular pedido de tomada de decisão apoiada, a pessoa com deficiência e os apoiadores devem apresentar termo em que constem os limites do apoio a ser oferecido e os compromissos dos apoiadores, inclusive o prazo de vigência do acordo e o respeito à vontade, aos direitos e aos interesses da pessoa que devem apoiar.      (Incluído pela Lei nº 13.146, de 2015)    (Vigência)\n\n§ 2 o O pedido de tomada de decisão apoiada será requerido pela pessoa a ser apoiada, com indicação expressa das pessoas aptas a prestarem o apoio previsto no caput deste artigo.      (In

## Load Model

In [9]:
llama3 = ChatGroq(
    temperature=1,
    model_name="llama3-70b-8192",
    api_key=getpass("API Groq"))

# Test
llama3.invoke("What is the capital of France?")

API Groq··········


AIMessage(content='The capital of France is Paris.', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 17, 'total_tokens': 24, 'completion_time': 0.019663548, 'prompt_time': 0.007800823, 'queue_time': None, 'total_time': 0.027464371}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_abd29e8833', 'finish_reason': 'stop', 'logprobs': None}, id='run-6ac3bc53-bbc3-4576-b51d-92e725c4ed54-0')

# Baseline 0

Sem passar trechos da lei - conhecimento da lei apenas do treinamento do modelo

In [11]:
baseline0_prompt = """
Using your knowledge of the Brazilian legal system, specially the Brazilian Civil Code
or Código Civil Brasileiro, your task is create questions in which solution could be find
in the articles described in LIVRO IV and LIVRO V of Parte Especial of the Código Civil
Brasileiro.
It would be great if your question can be answered by more then one articles and using articles
from both books.
In other to contextualize your questions, create situations where the solution can be found
in the articles.
Also, generate a short answers to your questions.
Finally, highlight the articles used to answer the questions in the designed field.
Answer in Brazilian Portuguese.
"""

# Define your desired data structure.
class OutputJSON(BaseModel):
    situacao: str = Field(description="situation in which the answer could be found")
    pergunta: str = Field(description="question that could be answered by articles")
    resposta: str = Field(description="short answer to the question")
    context: str = Field(description="List of articles that answer the question")

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=OutputJSON)

prompt = PromptTemplate(
    template=baseline0_prompt + "\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
chain = prompt | llama3 | parser
chain.invoke({"query": "Make a question"})

{'situacao': 'Maria e João são casados sob o regime de separação de bens. Eles adquiriram uma casa juntos, mas somente o nome de Maria está no registro de propriedade. Depois de uma discussão, João desejou saber se ele tem direito à metade da casa.',
 'pergunta': 'João tem direito à metade da casa?',
 'resposta': 'Sim, pois o Código Civil estabelece que os bens adquiridos durante o casamento são considerados como sendo de ambos os cônjuges, independentemente de quem está no registro de propriedade.',
 'context': 'Art. 1656 do Livro IV e Art. 1875 do Livro V'}

In [12]:
chain = prompt | llama3 | parser
output = chain.invoke({"query": "Make 30 questions with very different situations"})
print(output)

[{'situacao': 'Um contratante não cumpriu seu compromisso de entregar um produto dentro do prazo estabelecido.', 'pergunta': 'Existe algum mecanismo para exigir o cumprimento da obrigação?', 'resposta': 'Sim, o contratante pode exigir o cumprimento da obrigação.', 'context': 'Art. 475, Art. 476 do Código Civil'}, {'situacao': 'Dois amigos fizeram um contrato verbal para dividir os lucros de um negócio.', 'pergunta': 'Esse contrato é válido?', 'resposta': 'Não, o contrato deve ser escrito.', 'context': 'Art. 457, Art. 464 do Código Civil'}, {'situacao': 'Uma empresa não pagou um fornecedor dentro do prazo estabelecido.', 'pergunta': 'Quais são as consequências para a empresa?', 'resposta': 'A empresa pode ser responsável por perdas e danos.', 'context': 'Art. 395, Art. 404 do Código Civil'}, {'situacao': 'Um homem fez um contrato de trabalho com uma empresa.', 'pergunta': 'Quais são os direitos do trabalhador?', 'resposta': 'O trabalhador tem direito a salário, férias e outros direitos.

In [13]:
out_df = pd.DataFrame(output)
out_df.to_csv('baseline0_temp1.csv', index=False)
out_df

,situacao,pergunta,resposta,context
0,Um contratante não cumpriu seu compromisso de ...,Existe algum mecanismo para exigir o cumprimen...,"Sim, o contratante pode exigir o cumprimento d...","Art. 475, Art. 476 do Código Civil"
1,Dois amigos fizeram um contrato verbal para di...,Esse contrato é válido?,"Não, o contrato deve ser escrito.","Art. 457, Art. 464 do Código Civil"
2,Uma empresa não pagou um fornecedor dentro do ...,Quais são as consequências para a empresa?,A empresa pode ser responsável por perdas e da...,"Art. 395, Art. 404 do Código Civil"
3,Um homem fez um contrato de trabalho com uma e...,Quais são os direitos do trabalhador?,"O trabalhador tem direito a salário, férias e ...","Art. 442, Art. 443 do Código Civil"
4,Um contrato de venda foi feito entre duas partes.,Quais são os direitos do comprador?,O comprador tem direito à entrega do produto.,"Art. 481, Art. 482 do Código Civil"
5,Um contrato de serviço foi feito entre duas pa...,Quais são os deveres do prestador de serviço?,O prestador de serviço deve executar o serviço...,"Art. 492, Art. 493 do Código Civil"
6,Uma empresa não cumpriu um contrato de forneci...,Quais são as consequências para a empresa?,A empresa pode ser responsável por perdas e da...,"Art. 475, Art. 495 do Código Civil"
7,Dois amigos fizeram um contrato de empréstimo.,Quais são as obrigações do mutuário?,O mutuário deve pagar o empréstimo com juros.,"Art. 502, Art. 503 do Código Civil"
8,Um contrato de seguro foi feito entre uma empr...,Quais são os direitos do segurado?,O segurado tem direito à indenização em caso d...,"Art. 759, Art. 760 do Código Civil"
9,Um contrato de comissão foi feito entre uma em...,Quais são os direitos do comissário?,O comissário tem direito à comissão sobre as v...,"Art. 693, Art. 694 do Código Civil"


# Baseline 0 - Interações em PT_BR com o ChatGPT 3.5

User
```
Você conhece o código civil brasileiro?
```
ChatGPT
```
Sim, eu conheço o Código Civil Brasileiro. Ele é um conju...
```
User
```
Da parte especial do que se trata os livros IV e V?
```
ChatGPT
```
Na Parte Especial do Código Civil Brasileiro, os livros IV e V tratam...
```
User
```
Você poderia criar uma situação cuja solução reside no uso de artigos tanto do livro IV quanto do V?
E dessa situação gerar uma pergunta?
E dessa pergunta gerar uma resposta rápida?
E destacar em uma frase curta os artigos usados para responder a pergunta?
```
ChatGPT
```
Situação
Maria e João eram casados em regime de comunhão parcial ...
```
User
```
Muito bem, poderia fazer isso mais 29 vez com situações distintas?
```
ChatGPT
```
Claro, aqui estão mais algumas situações, perguntas, respostas rápidas e os artigos do Código Civil correspondentes:

Situação 1
Situação: João e Maria ...
```
*Obs: Criou 5 situações.*

User
```
Outras 5 explorando mais aspectos das aspectos das relações familiares, regimes de bens e sucessão.
```
ChatGPT
```
Claro, aqui estão mais cinco situações explorando diferentes aspectos das relações familiares, regimes de bens e sucessão no contexto do Código Civil Brasileiro:

Situação 6
Situação: Carlos e Ana estão ...
```

In [ ]:
gpt_df = pd.read_csv('/content/Baseline 0 - Interações em PT_BR com o ChatGPT 3.5 - Página1.csv')
gpt_df

,Situação,Pergunta,Resposta,Contexto
0,Maria e João eram casados em regime de comunhã...,Maria se casando novamente afeta os direitos d...,O novo casamento de Maria não altera os direit...,"Artigos 1.829, 1.832 e 1.845 do Código Civil."
1,João e Maria vivem em união estável há 10 anos...,Como será feita a partilha dos bens de João en...,"Em caso de falecimento sem testamento, Maria, ...","Artigos 1.725 (união estável), 1.829 (ordem de..."
2,"Roberto, ao falecer, deixa um imóvel como bem ...",Os herdeiros podem vender o imóvel considerado...,O bem de família é protegido por lei contra a ...,Artigos 1.711 (bem de família) e 1.715 (hipóte...
3,"Maria, casada com João sob o regime de comunhã...",Maria pode vender o imóvel adquirido por ela a...,"No regime de comunhão parcial de bens, os bens...",Artigos 1.667 (regime de comunhão parcial de b...
4,Ana recebeu uma doação de seu pai antes de se ...,A doação recebida por Ana de seu pai antes do ...,"Não, a doação recebida por Ana antes do casame...",Artigos 1.659 (comunicação dos bens no regime ...
5,Carlos e Ana estão em processo de divórcio apó...,Como será feita a partilha do imóvel adquirido...,O imóvel adquirido durante o casamento por Car...,Artigos 1.658 (partilha de bens no regime de c...
6,Joaquim e Helena vivem em união estável há 5 a...,Como será garantida a pensão alimentícia para ...,A pensão alimentícia para o filho menor de ida...,"Artigos 1.725 (união estável), 1.686 (pensão a..."
7,"Pedro é pai de três filhos, sendo dois do prim...",Como Pedro pode garantir que um dos filhos do ...,"Pedro pode estabelecer um testamento, especifi...",Artigos 1.846 (testamento) e 1.789 (partilha d...
8,"Marta e Luís vivem juntos há 20 anos, sem form...",Luís tem direito a herdar parte dos bens de Ma...,Luís poderá pleitear seus direitos na condição...,"Artigos 1.723 (união estável) e 1.725, § 1º (d..."
9,Sofia recebeu de seus avós uma casa como heran...,Como será feita a partilha da casa recebida po...,A casa recebida por Sofia como herança antes d...,Artigos 1.659 (comunicação dos bens no regime ...


In [ ]:
pd.read_csv('/content/Baseline 0 - Interações em PT_BR com o ChatGPT 3.5 - Pergunta Unica 1.csv')

,Situação,Pergunta,Resposta,Contexto,Prompt
0,Reconhecimento de Paternidade,Um filho pode exigir judicialmente o reconheci...,"Sim, o filho pode exigir judicialmente o recon...",Art. 1.607 e Art. 1.609.,Você poderia criar uma lista de 20 situação cu...
1,Guarda Compartilhada,A guarda compartilhada pode ser estabelecida m...,"Sim, a guarda compartilhada pode ser determina...","Art. 1.583, § 2º e Art. 1.584, § 2º.",Você poderia criar uma lista de 20 situação cu...
2,Regime de Comunhão Parcial de Bens,"No regime de comunhão parcial de bens, os bens...","Não, no regime de comunhão parcial de bens, ap...",Art. 1.658 e Art. 1.659.,Você poderia criar uma lista de 20 situação cu...
3,União Estável,União estável gera direitos e deveres semelhan...,"Sim, a união estável gera direitos e deveres s...",Art. 1.723 e Art. 1.725.,Você poderia criar uma lista de 20 situação cu...
4,Doação de Bens no Casamento,A doação de bens feita por um cônjuge a outro ...,"Sim, pode ser anulada se houver fraude contra ...",Art. 1.659 e Art. 550.,Você poderia criar uma lista de 20 situação cu...
5,Separação de Fato e Herança,A separação de fato afeta o direito à herança ...,"Não, a separação de fato não afeta o direito à...",Art. 1.830.,Você poderia criar uma lista de 20 situação cu...
6,Testamento e Legítima,É possível um testamento dispor sobre toda a h...,"Não, o testamento só pode dispor sobre metade ...",Art. 1.846 e Art. 1.789.,Você poderia criar uma lista de 20 situação cu...
7,Filiação Socioafetiva,A filiação socioafetiva pode gerar direitos su...,"Sim, a filiação socioafetiva pode gerar direit...",Art. 1.593 e Art. 1.596.,Você poderia criar uma lista de 20 situação cu...
8,Pacto Antenupcial,É possível modificar o regime de bens após o c...,"Sim, mediante autorização judicial e desde que...","Art. 1.639, § 2º.",Você poderia criar uma lista de 20 situação cu...
9,Herança de Convivente em União Estável,O convivente em união estável tem direito à he...,"Sim, o convivente tem direito à herança, conco...",Art. 1.790.,Você poderia criar uma lista de 20 situação cu...


# Baseline Adding Sample Article

In [ ]:
baseline0_prompt = """
Using your knowledge of the Brazilian legal system, specially the Brazilian Civil Code
or Código Civil Brasileiro, your task is create questions in which solution could be find
in the articles described in LIVRO IV and LIVRO V of Parte Especial of the Código Civil
Brasileiro.
To help you i have provided two sample groups of articles, one for each group.
It would be great if your question can be answered by more then one articles and using articles
from both books.
In other to contextualize your questions, create situations where the solution can be found
in the articles. Be creative and use different articles.
Also, generate a short answers to your questions.
Finally, highlight only the articles used to answer the questions in the designed field.
Answer in Brazilian Portuguese.
"""

# Define your desired data structure.
class OutputJSON(BaseModel):
    situacao: str = Field(description="situation in which the answer could be found")
    pergunta: str = Field(description="question that could be answered by articles")
    resposta: str = Field(description="short answer to the question")
    context: str = Field(description="List of articles that answer the question")

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=OutputJSON)

prompt = PromptTemplate(
    template=baseline0_prompt + "\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
chain = prompt | llama3 | parser
query = """
Make a question considering the articles below:
LIVRO IV:
{iv_articles}
LIVRO V:
{v_articles}
""".format(
    iv_articles=''.join(random.sample(iv_articles, 1)[0]).strip(),
    v_articles=''.join(random.sample(v_articles, 1)[0]).strip())
chain.invoke({"query": query})

{'situacao': 'Maria e João vivem juntos há 5 anos e têm um filho de 3 anos. Eles nunca se casaram, mas sempre viveram como se casados. Maria faleceu recentemente e deixou uma herança para João e o filho.',
 'pergunta': 'João pode herdar a propriedade de Maria, mesmo não estando casado com ela?',
 'resposta': 'Sim, João pode herdar a propriedade de Maria, pois eles viviam em união estável e têm um filho em comum.',
 'context': 'Art. 1.723, Art. 1.725, Art. 1.798, Art. 1.799'}

In [ ]:
query

'\nMake a question considering the articles below:\nLIVRO IV:\nArt. 1.723. É reconhecida como entidade familiar a união estável entre o homem e a mulher, configurada na convivência pública, contínua e duradoura e estabelecida com o objetivo de constituição de família.\n\n§ 1 o A união estável não se constituirá se ocorrerem os impedimentos do art. 1.521 ; não se aplicando a incidência do inciso VI no caso de a pessoa casada se achar separada de fato ou judicialmente.\n\n§ 2 o As causas suspensivas do art. 1.523 não impedirão a caracterização da união estável.\n\n  Art. 1.724. As relações pessoais entre os companheiros obedecerão aos deveres de lealdade, respeito e assistência, e de guarda, sustento e educação dos filhos.\n\n  Art. 1.725. Na união estável, salvo contrato escrito entre os companheiros, aplica-se às relações patrimoniais, no que couber, o regime da comunhão parcial de bens.\n\n  Art. 1.726. A união estável poderá converter-se em casamento, mediante pedido dos companheiros

In [ ]:
# samples_iv = []
# samples_v = []
# outputs = []
for _ in range(25-24):
    sample_iv = ''.join(random.sample(iv_articles, 1)[0]).strip()
    sample_v = ''.join(random.sample(v_articles, 1)[0]).strip()
    query = """
Make a question considering the articles below:
LIVRO IV:
{iv_articles}
LIVRO V:
{v_articles}
""".format(iv_articles=sample_iv, v_articles=sample_v)
    try:
        output = chain.invoke({"query": query})
    except:
        continue
    samples_iv.append(sample_iv)
    samples_v.append(sample_v)
    outputs.append(output)
    print(output)

{'situacao': 'Maria e João vivem juntos há 5 anos e decidiram fazer um testamento conjunto. Eles querem saber se podem incluir a legítima dos herdeiros necessários no testamento.', 'pergunta': 'É possível incluir a legítima dos herdeiros necessários no testamento?', 'resposta': 'Não', 'context': 'Art. 1.857, § 1o; Art. 1.723'}


In [ ]:
out_df = pd.DataFrame(outputs)
out_df.to_csv('baseline1.csv', index=False)
out_df

,situacao,pergunta,resposta,context
0,"Um pai, que tem dois filhos menores, é condena...",O pai perde o poder familiar em virtude da con...,Sim,"Art. 1.637, parágrafo único, e Art. 1.638"
1,Maria e João são casados em regime de separaçã...,Maria pode dispor de sua propriedade em seu te...,"Sim, Maria pode dispor de sua propriedade em s...","Art. 1.687, Art. 1.857, Art. 1.858"
2,"Um casal, João e Maria, decidiram se casar e e...",Quais são os direitos de Maria sobre a fazenda...,Maria tem direitos sobre a fazenda pois contri...,"Art. 1.687, Art. 1.688, Art. 1.923, Art. 1.924"
3,"Um homem, que é pai de um filho havido fora do...",Como o pai pode reconhecer seu filho em seu te...,O pai pode reconhecer seu filho em seu testame...,"Art. 1.609, III; Art. 1.610"
4,"João e Maria querem se casar, mas há uma dúvid...",O que deve ser feito se houver impugnação do o...,A habilitação será submetida ao juiz.,"Art. 1.526, Parágrafo único, e Art. 2.027"
5,"Maria e João, casados há 10 anos, decidem se d...","Como a guarda dos filhos será decidida, consid...",De acordo com o Art. 1.584 do Código Civil Bra...,"Art. 1.584, Art. 1.583, § 2º, Art. 1.585."
6,Um pai e uma mãe estão em processo de separaçã...,Quem exercerá o poder familiar sobre os filhos...,"Ambos os pais, conjuntamente, ou um deles, exc...","Art. 1.630, Art. 1.631, Art. 1.632"
7,Um pai deixa um testamento nomeando três tutor...,Pode o condenado exercer a tutela?,Não,"Art. 1.735, IV do Código Civil Brasileiro"
8,"Um pai, após se divorciar, casa novamente e te...",O pai perde o poder familiar sobre os filhos d...,"Não, o pai não perde o poder familiar sobre os...","Art. 1.636 do Código Civil Brasileiro, Art. 2...."
9,"Um homem viúvo deseja se casar novamente, mas ...",Pode o homem viúvo se casar novamente sem faze...,Não,"Art. 1.523, I"


In [ ]:
len(outputs)

9